In [1]:


import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd



![EuroSAT overview image](data/eurosat_overview.jpg)

#### The EuroSAT dataset

In this assignment, you will use the [EuroSAT dataset](https://github.com/phelber/EuroSAT). It consists of 27000 labelled Sentinel-2 satellite images of different land uses: residential, industrial, highway, river, forest, pasture, herbaceous vegetation, annual crop, permanent crop and sea/lake. For a reference, see the following papers:
- Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
- Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.

Your goal is to construct a neural network that classifies a satellite image into one of these 10 classes, as well as applying some of the saving and loading techniques you have learned in the previous sessions.

#### Import the data

The dataset you will train your model on is a subset of the total data, with 4000 training images and 1000 testing images, with roughly equal numbers of each class. The code to import the data is provided below.

In [2]:

def load_eurosat_data():
    data_dir = 'data/'
    x_train = np.load(os.path.join(data_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
    x_test  = np.load(os.path.join(data_dir, 'x_test.npy'))
    y_test  = np.load(os.path.join(data_dir, 'y_test.npy'))
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_eurosat_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

#### Build the neural network model

You can now construct a model to fit to the data. Using the Sequential API, build your model according to the following specifications:

* The model should use the input_shape in the function argument to set the input size in the first layer.
* The first layer should be a Conv2D layer with 16 filters, a 3x3 kernel size, a ReLU activation function and 'SAME' padding. Name this layer 'conv_1'.
* The second layer should also be a Conv2D layer with 8 filters, a 3x3 kernel size, a ReLU activation function and 'SAME' padding. Name this layer 'conv_2'.
* The third layer should be a MaxPooling2D layer with a pooling window size of 8x8. Name this layer 'pool_1'.
* The fourth layer should be a Flatten layer, named 'flatten'.
* The fifth layer should be a Dense layer with 32 units, a ReLU activation. Name this layer 'dense_1'.
* The sixth and final layer should be a Dense layer with 10 units and softmax activation. Name this layer 'dense_2'.

In total, the network should have 6 layers.

In [3]:


def get_new_model(input_shape):
    """
    This function should build a Sequential model according to the above specification. Ensure the 
    weights are initialised by providing the input_shape argument in the first layer, given by the
    function argument.
    Your function should also compile the model with the Adam optimiser, sparse categorical cross
    entropy loss function, and a single accuracy metric.
    """
    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = (3,3), padding = 'SAME', activation = 'relu', name = 'cov_1', input_shape =(input_shape), ))
    model.add(Conv2D(8, kernel_size = (3,3), padding = 'SAME', activation = 'relu', name = 'cov_2'))
    model.add(MaxPooling2D(pool_size=(8,8), name= 'pool_1'))
    model.add(Flatten())
    model.add(Dense(32, activation = 'relu', name = 'dense_1'))
    model.add(Dense(10, activation= 'softmax', name = 'dense_2'))
    
    
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    
    return model
    
    

#### Compile and evaluate the model

In [4]:


model = get_new_model(x_train[0].shape)

In [5]:


def get_test_accuracy(model, x_test, y_test):
    """Test model classification accuracy"""
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [6]:


model.summary()
get_test_accuracy(model, x_test, y_test)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cov_1 (Conv2D)               (None, 64, 64, 16)        448       
_________________________________________________________________
cov_2 (Conv2D)               (None, 64, 64, 8)         1160      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 8, 8, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                16416     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 18,354
Trainable params: 18,354
Non-trainable params: 0
____________________________________________________

#### Create checkpoints to save model during training, with a criterion


In [1]:


def get_checkpoint_every_epoch():
    """
    This function should return a ModelCheckpoint object that:
    - saves the weights only at the end of every epoch
    - saves into a directory called 'checkpoints_every_epoch' inside the current working directory
    - generates filenames in that directory like 'checkpoint_XXX' where
      XXX is the epoch number formatted to have three digits, e.g. 001, 002, 003, etc.
    """
    checkpoint_every_epoch = ModelCheckpoint(filepath='checkpoints_every_epoch/checkpoint_{epoch}',
                                            save_weights_only=True,
                                            frequency = 'epochs',
                                            )
    
    
    return checkpoint_every_epoch
    


def get_checkpoint_best_only():
    """
    This function should return a ModelCheckpoint object that:
    - saves only the weights that generate the highest validation (testing) accuracy
    - saves into a directory called 'checkpoints_best_only' inside the current working directory
    - generates a file called 'checkpoints_best_only/checkpoint' 
    """
    checkpoints_best_only = ModelCheckpoint(filepath = 'checkpoints_best_only/checkpoint',
                                           save_weights_only= True,
                                           save_best_only= True,
                                           monitor='val_accuracy',
                                           mode = 'max',
                                           verbose = False) 
    
    
    return checkpoints_best_only
    
    

In [8]:


def get_early_stopping():
    """
    This function should return an EarlyStopping callback that stops training when
    the validation (testing) accuracy has not improved in the last 3 epochs.
    HINT: use the EarlyStopping callback with the correct 'monitor' and 'patience'
    """
    
    earlystopping = EarlyStopping(monitor='val_accuracy',
                                 patience=3)
    
    
    return earlystopping
    
    

In [9]:

checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

#### Train model using the callbacks

In [10]:


callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

Train on 4000 samples, validate on 1000 samples
Epoch 1/50
4000/4000 [==============================] - 87s 22ms/sample - loss: 2.0257 - accuracy: 0.2282 - val_loss: 1.6489 - val_accuracy: 0.3910
Epoch 2/50
4000/4000 [==============================] - 87s 22ms/sample - loss: 1.4754 - accuracy: 0.4450 - val_loss: 1.4695 - val_accuracy: 0.4150
Epoch 3/50
4000/4000 [==============================] - 86s 22ms/sample - loss: 1.2923 - accuracy: 0.5110 - val_loss: 1.3408 - val_accuracy: 0.4830
Epoch 4/50
4000/4000 [==============================] - 85s 21ms/sample - loss: 1.2297 - accuracy: 0.5495 - val_loss: 1.2494 - val_accuracy: 0.5340
Epoch 5/50
4000/4000 [==============================] - 85s 21ms/sample - loss: 1.1490 - accuracy: 0.5845 - val_loss: 1.2361 - val_accuracy: 0.5570
Epoch 6/50
4000/4000 [==============================] - 86s 21ms/sample - loss: 1.0674 - accuracy: 0.6225 - val_loss: 1.1738 - val_accuracy: 0.5700
Epoch 7/50
4000/4000 [==============================] - 86s 22ms

#### Create new instance of model and load on both sets of weights



In [19]:
! ls -lh checkpoints_every_epoch

total 6.4M
-rw-r--r-- 1 jovyan users   83 Dec 29 17:30 checkpoint
-rw-r--r-- 1 jovyan users 219K Dec 29 17:02 checkpoint_10.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 29 17:02 checkpoint_10.index
-rw-r--r-- 1 jovyan users 219K Dec 29 17:04 checkpoint_11.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 29 17:04 checkpoint_11.index
-rw-r--r-- 1 jovyan users 219K Dec 29 17:05 checkpoint_12.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 29 17:05 checkpoint_12.index
-rw-r--r-- 1 jovyan users 219K Dec 29 17:07 checkpoint_13.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 29 17:07 checkpoint_13.index
-rw-r--r-- 1 jovyan users 219K Dec 29 17:08 checkpoint_14.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 29 17:08 checkpoint_14.index
-rw-r--r-- 1 jovyan users 219K Dec 29 17:10 checkpoint_15.data-00000-of-00001
-rw-r--r-- 1 jovyan users 2.0K Dec 29 17:10 checkpoint_15.index
-rw-r--r-- 1 jovyan users 219K Dec 29 17:11 checkpoint_16.data-00000-of-00001
-rw-

In [26]:


def get_model_last_epoch(model):
    """
    This function should create a new instance of the CNN you created earlier,
    load on the weights from the last training epoch, and return this model.
    """
    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir='checkpoints_every_epoch'))
    
    
    return model
    
    
    
    
    
    
    
    
def get_model_best_epoch(model):
    """
    This function should create a new instance of the CNN you created earlier, load 
    on the weights leading to the highest validation accuracy, and return this model.
    """
    
    model.load_weights('checkpoints_best_only/checkpoint')
    return model
    
    

In [27]:


model_last_epoch = get_model_last_epoch(get_new_model(x_train[0].shape))
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('')
print('Model with best epoch weights:')
get_test_accuracy(model_best_epoch, x_test, y_test)

Model with last epoch weights:
accuracy: 0.715

Model with best epoch weights:
accuracy: 0.732


#### Load, from scratch, a model trained on the EuroSat dataset.

In your workspace, you will find another model trained on the `EuroSAT` dataset in `.h5` format. This model is trained on a larger subset of the EuroSAT dataset and has a more complex architecture. The path to the model is `models/EuroSatNet.h5`. See how its testing accuracy compares to your model!

In [28]:


def get_model_eurosatnet():
    """
    This function should return the pretrained EuroSatNet.h5 model.
    """
    model = load_model('models/EuroSatNet.h5')
    
    return model
    
    

In [29]:


model_eurosatnet = get_model_eurosatnet()
model_eurosatnet.summary()
get_test_accuracy(model_eurosatnet, x_test, y_test)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 16)        6416      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 32, 32, 16)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 32, 32, 16)        2320      
_________________________________________________________________
conv_4 (Conv2D)              (None, 32, 32, 16)        6416      
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 16, 16, 16)        0         
_________________________________________________________________
conv_5 (Conv2D)              (None, 16, 16, 16)      